# Análisis de Datos del Titanic
Este notebook replica el flujo del notebook de Spotify aplicado al dataset `titanic.csv`.
Se enfatiza el análisis: cajas y bigotes, boxplots por categoría, correlaciones, scatter plots con línea de tendencia y mapas de calor.

Usaremos Pandas, Seaborn y Matplotlib. Después de cada visualización hay interpretación y conclusiones cortas.

# 🛳 Análisis de Datos del Titanic
Notebook adaptado del estilo del profesor. Se incluyen visualizaciones y explicaciones interpretativas después de cada gráfico.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
sns.set(style="whitegrid")
%matplotlib inline

In [ ]:
data_path = Path('titanic.csv')
if data_path.exists():
    df = pd.read_csv(data_path)
    print(f'Archivo cargado desde: {data_path}')
    display(df.head())
else:
    raise FileNotFoundError(f'No se encontró titanic.csv en: {data_path.resolve()}')

In [ ]:
print('Columnas:', df.columns.tolist())
print('\nTipos:' )
display(df.dtypes)
print('\nValores nulos por columna:' )
display(df.isna().sum())

In [ ]:
df_clean = df.copy()
for col in ['Age','Fare','SibSp','Parch','Pclass','Survived']:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
if 'Fare' in df_clean.columns:
    df_clean['Fare_pos'] = df_clean['Fare'].replace(0, np.nan)

In [ ]:
plt.figure(figsize=(12,6))
df_plot = df_clean.copy()
df_plot['Fare_plot'] = df_plot['Fare_pos'].fillna(df_plot['Fare'])
sns.boxplot(data=df_plot, x='Pclass', y='Fare_plot', hue='Sex')
plt.yscale('log')
plt.title('Distribución de Fare por Clase y Sexo (escala log)')
plt.ylabel('Fare (log scale)')
plt.show()

Interpretación:
- La variable `Fare` presenta outliers y mayor dispersión para la primera clase.
- Las comparaciones por sexo dentro de cada clase pueden revelar diferencias de acceso a tickets o combinaciones de reserva.

## Correlaciones y mapa de calor
Calcularemos la matriz de correlación entre variables numéricas y la visualizaremos mediante un heatmap.

In [ ]:
num_cols = [c for c in ['Age','Fare','SibSp','Parch','Survived','Pclass'] if c in df_clean.columns]
corr = df_clean[num_cols].corr()
display(corr)

In [ ]:
plt.figure(figsize=(7,6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de calor de correlaciones (variables numéricas)')
plt.show()

Interpretación rápida:
- Observa correlaciones con `Survived` para identificar variables con asociación positiva o negativa.
- Recuerda la diferencia entre correlación y causalidad; controlar por `Pclass` y `Sex` en análisis posteriores.

## 3️⃣ Scatter plots con línea de tendencia
Mostraremos pares interesantes (Age vs Fare, Fare vs Survived, SibSp vs Parch, Age vs Survived) y el coeficiente r de Pearson.

In [ ]:
pairs = [('Age','Fare'), ('Fare','Survived'), ('SibSp','Parch'), ('Age','Survived')]
pairs = [p for p in pairs if p[0] in df_clean.columns and p[1] in df_clean.columns]
fig, axes = plt.subplots(2,2, figsize=(14,10))
axes = axes.flatten()
for ax, (x,y) in zip(axes, pairs):
    sub = df_clean[[x,y]].dropna()
    if len(sub) < 2:
        ax.text(0.5,0.5,'Datos insuficientes', ha='center')
        ax.set_axis_off()
        continue
    r = sub[x].corr(sub[y])
    sns.regplot(data=sub, x=x, y=y, ax=ax, scatter_kws={'alpha':0.4,'s':20}, line_kws={'color':'red'})
    ax.set_title(f'{x} vs {y} (r={r:.2f})')
    if x == 'Survived' or y == 'Survived':
        ax.set_yticks([0,1])
plt.tight_layout()
plt.show()

Interpretación:
- `Fare` vs `Survived`: r positiva suele indicar que tarifas mayores se asociaron con mayor supervivencia (efecto de clase).
- `Age` vs `Survived`: revisar estratos por `Pclass` y `Sex` para entender mejor el patrón.

In [ ]:
pairs = []
cols = corr.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        a, b = cols[i], cols[j]
        pairs.append((a, b, corr.loc[a,b]))
pairs_sorted = sorted(pairs, key=lambda x: abs(x[2]), reverse=True)
print('Top 5 pares por |r|:')
for a,b,r in pairs_sorted[:5]:
    signo = 'positiva' if r>0 else 'negativa'
    print(f' - {a} vs {b}: r={r:.2f} ({signo})')

# 5 oraciones interpretativas
if pairs_sorted:
    top = pairs_sorted[0]
    print('Cinco oraciones interpretativas:')
    print(f"1) El par con mayor correlación absoluta es {top[0]} vs {top[1]} (r={top[2]:.2f}), indicando una asociación {'positiva' if top[2]>0 else 'negativa' }.")
    influence = {v: sum(abs(corr[v].drop(v))) for v in cols}
    var_most = max(influence.items(), key=lambda x: x[1])
    var_least = min(influence.items(), key=lambda x: x[1])
    print(f"2) La variable más influyente (suma de |r|) es {var_most[0]} con suma={var_most[1]:.2f}.")
    print(f"3) La variable menos correlacionada es {var_least[0]} con suma={var_least[1]:.2f}.")
    if 'Survived' in cols:
        print(f"4) Correlaciones con 'Survived': {[ (c, float(corr.loc['Survived', c])) for c in cols if c!='Survived' ]}")
    print('5) Recordar que correlación no implica causalidad; controlar por Pclass y Sex en análisis posteriores.')

## Conclusiones finales
- Hallazgos principales: diferencias por clase en `Fare` y `Age`, asociaciones con `Survived` y presencia de outliers en `Fare`.
- Recomendaciones: análisis estratificado por `Pclass` y `Sex`, imputación cuidadosa de `Age` y modelos de regresión logística para estudiar factores de supervivencia.